# Merging Data

We often want to combine data stored in multiple different sources into a single representation for analysis or modelling. In this notebook, we look at how we can **merge** data using Pandas DataFrames.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Data Loading 

We will work with records for university students and programmes, which are split across two separate CSV files:
- *student_details.csv*: A master list of students with their unique student ID, first name, surname, gender, and university email.
- *student_enrolment.csv*: Enrolment records for students, indicating their study level, school, and programme year linked by student ID.

In [2]:
# read the first dataset
df_details = pd.read_csv("data/student_details.csv")
print(f"DataFrame has {df_details.shape[0]} rows and {df_details.shape[1]} columns")
print(f"Columns: {df_details.columns.tolist()}")
df_details.head(10)

DataFrame has 50 rows and 5 columns
Columns: ['student_id', 'firstname', 'lastname', 'gender', 'email']


,student_id,firstname,lastname,gender,email
0,250001,Ciara,Walsh,F,ciara.walsh@uni.ie
1,250002,Micheal,Connolly,M,micheal.connolly@uni.ie
2,250003,Xu,Li,M,xu.li@uni.ie
3,250004,Sia,Khalfaoui,F,sia.khalfaoui@uni.ie
4,250005,Orlaith,Gallagher,F,orlaith.gallagher@uni.ie
5,250006,Nora,Curran,F,nora.curran@uni.ie
6,250007,Sophia,Kim,F,sophia.kim@uni.ie
7,250008,Dara,Ryan,M,dara.ryan@uni.ie
8,250009,Roisin,Ward,F,roisin.ward@uni.ie
9,250010,Brendan,Sheehan,M,brendan.sheehan@uni.ie


In [3]:
# read the second dataset
df_enrol = pd.read_csv("data/student_enrolment.csv")
print(f"DataFrame has {df_enrol.shape[0]} rows and {df_enrol.shape[1]} columns")
print(f"Columns: {df_enrol.columns.tolist()}")
df_enrol.head(10)

DataFrame has 45 rows and 4 columns
Columns: ['student_id', 'level', 'school', 'year']


,student_id,level,school,year
0,250004,Undergraduate,Mathematics,3
1,250008,Postgraduate Taught,Mathematics,1
2,250010,Undergraduate,Computer Science,1
3,250041,Undergraduate,Mathematics,1
4,250011,Undergraduate,Computer Science,2
5,250044,Undergraduate,Computer Science,2
6,250028,Postgraduate Research,Mathematics,4
7,250039,Postgraduate Research,Chemistry,2
8,250005,Undergraduate,Physics,3
9,250025,Undergraduate,Chemistry,1


## Merging Data - Inner Joins

Merging two DataFrames in Pandas involves matching rows based on one or more key columns and combining their associated data into a single unified DataFrame. We do this using the `pd.merge()` function.

The most common type of merge in Pandas is an **inner join**. This operation keeps only rows that have matching keys in both DataFrames.

In the case of our data, we join the two DataFrames based on the column `student_id` in each case, so only students who are present in both files will appear in the result. 

Note that when we call the `pd.merge()` function here, the argument `how="inner"` indicates that an inner join is to be performed.

In [4]:
# perform inner join on student_id
df_merged1 = pd.merge(
    df_details, df_enrol,
    how="inner",
    on="student_id"
)

print(f"Merged DataFrame has {df_merged1.shape[0]} rows and {df_merged1.shape[1]} columns")
print(f"Merged Columns: {df_merged1.columns.tolist()}")

Merged DataFrame has 45 rows and 8 columns
Merged Columns: ['student_id', 'firstname', 'lastname', 'gender', 'email', 'level', 'school', 'year']


Notice that our new merged DataFrame only contains rows whose `student_id` appears in both input DataFrames, so any students without a matching enrolment are excluded from the result.

In [5]:
# check the original student IDs from the details table
details_student_ids = set(df_details["student_id"])
print(f"Count of original students: {len(details_student_ids)}")

# check the student IDs from the merged table
merged1_student_ids = set(df_merged1["student_id"])
print(f"Count of enrolled students: {len(merged1_student_ids)}")

# which students aren't enrolled?
print("Missing students:")
details_student_ids.difference(merged1_student_ids)

Count of original students: 50
Count of enrolled students: 45
Missing students:


{250002, 250012, 250016, 250037, 250042}

We could now perform characterisation of the data in our merged DataFrame.

For instance, we could look at gender balance by programme level or by school:

In [6]:
# cross tabulation of level and gender
pd.crosstab(df_merged1["level"], df_merged1["gender"], margins=True)

gender,F,M,All
level,,,
Postgraduate Research,7,6,13
Postgraduate Taught,2,4,6
Undergraduate,15,11,26
All,24,21,45


In [7]:
# cross tabulation of school and gender
pd.crosstab(df_merged1["school"], df_merged1["gender"], margins=True)

gender,F,M,All
school,,,
Chemistry,5,2,7
Computer Science,10,10,20
Mathematics,4,6,10
Physics,5,3,8
All,24,21,45


## Merging Data - Outer Joins

An alternative merging strategy is an **outer join**. This includes all keys from both tables: matched rows where possible, and unmatched rows filled with missing values. This is useful for a completeness view, but it introduces missing values for non-matches.

Again, we will join the two DataFrames based on the column `student_id`. However, when we call the `pd.merge()` function now, the argument `how="outer"` indicates that an outer join is to be performed.

In [8]:
# perform outer join
df_merged2 = pd.merge(
    df_details, df_enrol,
    how="outer"
)

print(f"Merged DataFrame has {df_merged2.shape[0]} rows and {df_merged2.shape[1]} columns")
print(f"Merged Columns: {df_merged2.columns.tolist()}")

Merged DataFrame has 50 rows and 8 columns
Merged Columns: ['student_id', 'firstname', 'lastname', 'gender', 'email', 'level', 'school', 'year']


Notice now that our merged DataFrame now contains rows for all students in the original details DataFrame.

However, we do now have rows with missing values - i.e. students that are not enrolled on aany programme.

In [9]:
# check columns for missing values 
df_merged2.isna().sum()

student_id    0
firstname     0
lastname      0
gender        0
email         0
level         5
school        5
year          5
dtype: int64

In [10]:
# check which rows have missing values (i.e. students not enrolled)
df_merged2[df_merged2.isna().any(axis=1)]

,student_id,firstname,lastname,gender,email,level,school,year
1,250002,Micheal,Connolly,M,micheal.connolly@uni.ie,NaN,NaN,NaN
11,250012,Bob,Maguire,M,bob.maguire@uni.ie,NaN,NaN,NaN
15,250016,Fiona,O'Reilly,F,fiona.oreilly@uni.ie,NaN,NaN,NaN
36,250037,Alan,Higgins,M,alan.higgins@uni.ie,NaN,NaN,NaN
41,250042,Saoirse,Corcoran,F,saoirse.corcoran@uni.ie,NaN,NaN,NaN


We could decide to replace the missing values in some of these columns:

In [11]:
df_merged2["level"] = df_merged2["level"].fillna("Unknown")
df_merged2["school"] = df_merged2["school"].fillna("Not allocated")

Use frequency tables to check the counts for these columns, after filling missing values:

In [12]:
for col in ["level", "school"]:
    print(f"- Column: {col}")
    display(df_merged2[col].value_counts())
    print()

- Column: level


level
Undergraduate            26
Postgraduate Research    13
Postgraduate Taught       6
Unknown                   5
Name: count, dtype: int64


- Column: school


school
Computer Science    20
Mathematics         10
Physics              8
Chemistry            7
Not allocated        5
Name: count, dtype: int64